Docker image built with [Dockerfile](https://github.com/palmoreck/dockerfiles/blob/master/jupyterlab/kale/distributed/0.1/Dockerfile)

In [1]:
import math
import time

from dask.distributed import Client
import multiprocessing
from scipy.integrate import quad
from pytest import approx

In [2]:
f = lambda x: math.exp(-x**2)
a = 0
b = 1
n = 10**7
h_hat = (b-a)/n
p = multiprocessing.cpu_count()
n_subintervals_per_core = int(n/p)

In [3]:
def create_nodes_in_subintervals(my_id,a,b,h_hat,n,n_s_c):
    begin = my_id*n_s_c
    end = begin+n_s_c
    h_hat = (b-a)/n
    nodes = []
    for i in range(begin,end):
        nodes.append(a+(i+1/2.0)*h_hat)
    return nodes
def evaluate_f_in_nodes_and_sum(nodes,f):
    sum_res = 0
    for node in nodes:
        sum_res+=f(node)
    return sum_res

In [4]:
client = Client()
futures_nodes = client.map(create_nodes_in_subintervals,range(p),
                           **{'a':a,
                              'b':b,
                              "h_hat":h_hat,
                              'n':n,
                              "n_s_c":n_subintervals_per_core}
                            )
futures_evaluated = client.map(evaluate_f_in_nodes_and_sum, futures_nodes,
                            **{'f':f}
                            )
results_gather = client.gather(futures_evaluated)

In [6]:
res_rcf_distributed = h_hat*sum(results_gather)

In [8]:
print(res_rcf_distributed)

0.7468241328124374
